In [1]:
print("all okay")

all okay


In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

In [2]:
from dotenv import load_dotenv
import os

load_dotenv() 


# db_host = os.getenv("PINECONE_API_KEY")
# db_host
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_KEY

'ead9261a-2bb9-4afa-a1aa-056cc1d4a5e9'

In [3]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV =os.getenv("PINECONE_API_ENV")
PINECONE_API_ENV

'gcp-starter'

### Extract Data from the pdf

In [4]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="./*.pdf",loader_cls=PyPDFLoader)
    return loader.load()

extracted_data=load_pdf("../data/")

In [6]:
len(extracted_data)

637

### Create Text Chunks

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
docs = text_split(extracted_data)
print("length of my chunk:", len(docs))

length of my chunk: 7020


### Download the embedding models

In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

d:\Personal\GEN_AI\chatbot-llama2-\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))
# query_result

Length 384


### Initialize Pinecone

In [9]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV =os.getenv("PINECONE_API_ENV")
PINECONE_INDEX_NAME =os.getenv("PINECONE_INDEX_NAME")

PINECONE_INDEX_NAME,PINECONE_API_KEY

('test', 'ead9261a-2bb9-4afa-a1aa-056cc1d4a5e9')

In [11]:
from langchain_pinecone import PineconeVectorStore

index_name = "medical"

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [14]:
query = "What are Allergies"
docs = docsearch.similarity_search(query,k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'page': 130.0, 'source': '..\\data\\Medical_book.pdf'}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food add

In [15]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [17]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [18]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [20]:
# result=qa({"query": query})
result=qa.invoke(query)
print("Response : ", result["result"])

Response :  Allergies occur when the body's immune system overreacts to a harmless substance, such as pollen, dust, or a specific food. When an allergen enters the body, the immune system produces antibodies called IgE, which bind to the allergen and trigger the release of chemical mediators, such as histamine. This leads to an allergic reaction, which can cause symptoms such as sneezing, itching, swelling, and difficulty breathing.
